<a href="https://colab.research.google.com/github/Zhu-Pengming/Flora-Talks/blob/main/bert_base_chinese_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

首先，挂载Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


然后，使用pandas来读取你的Excel文件

In [ ]:
import pandas as pd

# 替换为你的实际文件路径
powder_df = pd.read_excel('/content/drive/MyDrive/Flora/cleaned_powder.xlsx')
rust_df = pd.read_excel('/content/drive/MyDrive/Flora/cleaned_rust.xlsx')


安装所需的库

In [ ]:
!pip install tensorflow
!pip install transformers


导入所需的库

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split


加载预训练的BERT模型和分词器

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = TFBertForSequenceClassification.from_pretrained('bert-base-chinese')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


首先，为每个数据集的样本分配一个标签

In [ ]:
powder_df['label'] = 0
rust_df['label'] = 1


将两个数据集合并为一个

In [ ]:
df = pd.concat([powder_df, rust_df])

df = df.applymap(lambda x: str(x) if not isinstance(x, str) else x)



将文本和标签转换为模型可以接受的格式

In [ ]:

sentences = df['评论内容'].values

sentences = sentences.tolist()
labels = df['label'].values




# 使用分词器对句子进行编码
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='tf')

# 将标签转换为Tensor
labels = tf.convert_to_tensor(labels)


from sklearn.model_selection import train_test_split

# 将inputs的各部分转换为numpy数组
input_ids = inputs['input_ids'].numpy()
token_type_ids = inputs['token_type_ids'].numpy()
attention_mask = inputs['attention_mask'].numpy()

# 将labels也转换为numpy数组
labels = labels.numpy()

# 使用train_test_split函数进行划分
train_input_ids, test_input_ids, train_labels, test_labels = train_test_split(input_ids, labels, test_size=0.2, random_state=42)
train_token_type_ids, test_token_type_ids = train_test_split(token_type_ids, test_size=0.2, random_state=42)
train_attention_mask, test_attention_mask = train_test_split(attention_mask, test_size=0.2, random_state=42)



训练模型

In [ ]:
from sklearn.preprocessing import LabelEncoder

# 创建标签编码器
le = LabelEncoder()

# 转换训练和测试标签
train_labels = le.fit_transform(train_labels)
test_labels = le.transform(test_labels)

# 创建训练和测试输入
train_inputs = [train_input_ids, train_token_type_ids, train_attention_mask]
test_inputs = [test_input_ids, test_token_type_ids, test_attention_mask]

# 编译模型
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_inputs, train_labels, epochs=3, validation_data=(test_inputs, test_labels))



Epoch 1/3
345/345 [==============================] - 548s 1s/step - loss: 0.6959 - accuracy: 0.5873 - val_loss: 0.7301 - val_accuracy: 0.5568
Epoch 2/3
345/345 [==============================] - 473s 1s/step - loss: 0.6999 - accuracy: 0.5471 - val_loss: 0.6740 - val_accuracy: 0.5539
Epoch 3/3
345/345 [==============================] - 474s 1s/step - loss: 0.6999 - accuracy: 0.5596 - val_loss: 0.7654 - val_accuracy: 0.5616


保存模型

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
model.save('/content/drive/MyDrive/Flora/model_Y')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


使用这个模型进行预测

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
!pip install numpy
import numpy as np


# 使用模型生成预测结果
predictions = model.predict(test_inputs)

logits = predictions.logits
print(logits.shape)

predicted_labels = np.argmax(logits, axis=1)
print(predicted_labels)


# 计算精确率
precision = precision_score(test_labels, predicted_labels, average='micro')

# 计算召回率
recall = recall_score(test_labels, predicted_labels, average='micro')

# 计算F1分数
f1 = f1_score(test_labels, predicted_labels, average='micro')

print('Precision: ', precision)
print('Recall: ', recall)
print('F1 Score: ', f1)



87/87 [==============================] - 36s 408ms/step
(2753, 2)
[1 1 1 ... 1 1 1]
Precision:  0.6018888485288776
Recall:  0.6018888485288776
F1 Score:  0.6018888485288776


In [5]:
!pip uninstall tensorflow
!pip install tf-nightly

import tensorflow as tf
from tensorflow import keras
import os

from google.colab import drive
drive.mount('/content/drive')


saved_model_dir = '/content/drive/MyDrive/Flora/model_Y'

try:
    loaded_model = tf.keras.models.load_model(saved_model_dir)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading the model: {e}")

# 创建一个转换器对象
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

# 尝试转换模型
try:
    tflite_model = converter.convert()
    print("Model converted successfully.")
except Exception as e:
    print(f"Error converting the model: {e}")

# 指定转换后的模型保存路径
tflite_model_path = os.path.join(os.path.dirname(saved_model_dir), 'model.tflite')

# 保存转换后的模型
try:
    with open(tflite_model_path, 'wb') as f:
        f.write(tflite_model)
    print(f"Model converted and saved at: {tflite_model_path}")
except Exception as e:
    print(f"Error saving the converted model: {e}")


Mounted at /content/drive
Error loading the model: File format not supported: filepath=/content/drive/MyDrive/Flora/model_Y. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(/content/drive/MyDrive/Flora/model_Y, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).
Model converted successfully.
Model converted and saved at: /content/drive/MyDrive/Flora/model.tflite
